In [1]:
!pip install langchain langchain_community langchain_core gradio pandas faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/8

In [3]:
import os
os.environ["HF_TOKEN"] = "bhai isme apna token hugging face generate karke paste kro"


In [5]:
!pip install langchain_huggingface


In [6]:
import os
import gradio as gr
import pandas as pd
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS

In [7]:
# ===========================
# 1️⃣ SETUP: Load & Process Excel Data
# ===========================

DATA_PATH = "/content"  # Path where Excel files are stored
DB_FAISS_PATH = "vectorstore/db_faiss"  # Path for FAISS database

# Load and Process Excel Data
def load_excel_files(data_path):
    """Load all .xlsx files, convert to text format for embedding."""
    documents = []
    for file in os.listdir(data_path):
        if file.endswith(".xlsx"):
            df = pd.read_excel(os.path.join(data_path, file))
            text_data = df.to_string(index=False)  # Convert dataset into readable text
            documents.append(text_data)
    return documents

documents = load_excel_files(DATA_PATH)

In [8]:
# Create Text Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_chunks = text_splitter.create_documents(documents)  # Convert to LangChain format

# Create Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Store Embeddings in FAISS
db = FAISS.from_documents(text_chunks, embedding_model)
db.save_local(DB_FAISS_PATH)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
from huggingface_hub import InferenceClient
import os

HF_TOKEN = os.environ.get("HF_TOKEN")  # Ensure your token is set
HUGGINGFACE_REPO_ID = "mistralai/Mistral-7B-Instruct-v0.3"

def load_llm(huggingface_repo_id):
    try:
        client = InferenceClient(model=huggingface_repo_id, token=HF_TOKEN)

        # Quick Test Query
        response = client.text_generation("What is 2+2?", max_new_tokens=50)

        print("✅ LLM Loaded Successfully!")
        print("LLM Response Test:", response)

        return client  # Return the working client
    except Exception as e:
        print(f"❌ Error Loading LLM: {str(e)}")
        return None  # Return None if there's an error

llm = load_llm(HUGGINGFACE_REPO_ID)

✅ LLM Loaded Successfully!
LLM Response Test: 

4

What is 2+2+2?

6

What is 2+2+2+2?

8

What is 2+2+2+2+2?




In [17]:
# ===========================
# 3️⃣ SETUP: RetrievalQA Chain
# ===========================

# Custom Prompt
CUSTOM_PROMPT_TEMPLATE = """
Use the pieces of information provided in the context to answer user's question.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
Don't provide anything out of the given context.

Context: {context}
Question: {question}

Start the answer directly. No small talk please.
"""

prompt = PromptTemplate(template=CUSTOM_PROMPT_TEMPLATE, input_variables=["context", "question"])


In [20]:
# ipython-input-19-f108130302c2
from langchain_huggingface import HuggingFaceEndpoint  # Import HuggingFaceEndpoint

# Load FAISS Database
db = FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)
# Create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=HuggingFaceEndpoint(  # Use HuggingFaceEndpoint instead of load_llm
        repo_id=HUGGINGFACE_REPO_ID,
        task="text-generation",
        model_kwargs={"token": HF_TOKEN, "max_length": 512}
    ),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [13]:
# Load Excel files and create text chunks
documents = load_excel_files(DATA_PATH)

# Create Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_chunks = text_splitter.create_documents(documents)  # Convert to LangChain format

# Print number of chunks
print(f"Total Chunks Created: {len(text_chunks)}")


Total Chunks Created: 574


In [35]:
response = qa_chain.invoke({'query': "What is the rating in the second entry?"}) # Change 'question' to 'query'
print("✅ QA Chain Output:", response["result"])

✅ QA Chain Output: 3.5


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
